<a href="https://colab.research.google.com/github/venkatchiranjeevireddy/Yardstick_assignment/blob/main/Yardstick_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 1: Conversation Management with Summarization (Groq API)

This notebook is my solution for **Task 1** of the assignment.  
The goal here is to show how we can:

Keep track of a conversation history (user ↔ assistant)  
Summarize the history every few turns to keep it short and clean  
Provide options to truncate history by number of turns or word count  
Use **Groq API** with **OpenAI SDK compatibility**  

I wrote this notebook step by step, so you can follow along like a story.  
Each code cell has an explanation written in plain English so anyone can understand what's happening.  




##Step 1: Install the required library

We only need the `openai` Python library to talk to Groq’s API.  
This step makes sure the library is available inside Google Colab.  


In [47]:
!pip install openai --quiet


##Step 2: Connect to Groq API

Here we connect to the Groq API using the OpenAI-compatible client.  
This way, we can use Groq just like we’d use OpenAI.


In [48]:
import openai
import os
from google.colab import userdata
api_key = userdata.get("GROQ_API_KEY")
# Configure OpenAI client for Groq API
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=api_key
)

## Step 3: Conversation History Setup
Here I create a list to store the whole conversation.  
Every message has two parts:
- **role** (user, assistant, or system)
- **content** (the actual text)
This is just like a chat app storing who said what.


In [49]:
# Initialize the conversation history as an empty list
conversation_history = []
def add_message(role, content):
    """Adds a new message to the conversation history."""
    conversation_history.append({"role": role, "content": content})
def get_conversation():
    """Return the current conversation history."""
    return conversation_history


##Step 4: Summarization Function

This function takes the entire conversation and asks the Groq model to **summarize** it.  
Why? Because if the conversation becomes too long, we don’t want to keep sending the entire text.  
Instead, we can keep a short version (summary) that still captures the meaning.  


In [50]:
def summarize_conversation(conversation):
    """Summarize the conversation using Groq API."""
    prompt = "Summarize the following conversation:\n\n"
    for msg in conversation:
        prompt += f"{msg['role'].capitalize()}: {msg['content']}\n"
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes conversations."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )
    summary = response.choices[0].message.content
    return summary


##Step 5: Truncation Options

Sometimes we don’t want the full conversation.  
Here I show two ways to cut it down:
1. **By turns** → Keep only the last N messages.  
2. **By word count** → Keep only up to N words (most recent words).  

This is super useful for keeping the chat light and efficient.


In [51]:
def truncate_by_turns(conversation, n):
    """Keep only the last n messages."""
    return conversation[-n:] if len(conversation) > n else conversation


In [52]:
def truncate_by_word_count(conversation, max_words):
    """Truncate the conversation to a maximum number of words."""
    total_words = 0
    truncated = []
    # Go through messages from last to first
    for msg in reversed(conversation):
        word_count = len(msg['content'].split())
        if total_words + word_count <= max_words:
            truncated.insert(0, msg)
            total_words += word_count
        else:
            break
    return truncated


##Step 6: Periodic Summarization

Instead of summarizing every single turn, we can do it **every k turns**.  
For example: summarize after every 3 turns.  
This way, the conversation stays fresh without losing too much detail.


In [53]:
def maybe_summarize(turn_count, k):
    """Check if summarization should happen after every k turns."""
    return turn_count % k == 0
def apply_periodic_summarization(conversation, turn_count, k=3):
    """Perform summarization every k turns and replace history with summary."""
    if maybe_summarize(turn_count, k):
        summary = summarize_conversation(conversation)
        return [{"role": "system", "content": f"Summary of conversation:\n{summary}"}]
    return conversation


In [54]:
import json

##Step 7: Example Conversation Demo

Now let’s test everything with a fake conversation.  
I made up some turns between a user and the assistant.  
Watch how the conversation history changes:  
- At normal turns, it just adds the message.  
- After every 3 turns, it replaces the history with a neat summary.  


In [55]:
# Reset conversation history
conversation_history = []
k = 3  # Summarize after every 3 turns
# Example conversation turns
messages = [
    ("user", "Hello!"),
    ("assistant", "Hi there! How can I help you?"),
    ("user", "I want to learn about AI."),
    ("assistant", "AI is the simulation of human intelligence in machines."),
    ("user", "Can you explain machine learning?"),
    ("assistant", "Machine learning allows systems to learn from data."),
]
# Simulate conversation turns
for i, (role, content) in enumerate(messages, start=1):
    add_message(role, content)
    print(f"\nTurn {i}: {role} says: {content}")
    conversation_history = apply_periodic_summarization(conversation_history, i, k)
    print("Current Conversation:")
    print(json.dumps(conversation_history, indent=2))



Turn 1: user says: Hello!
Current Conversation:
[
  {
    "role": "user",
    "content": "Hello!"
  }
]

Turn 2: assistant says: Hi there! How can I help you?
Current Conversation:
[
  {
    "role": "user",
    "content": "Hello!"
  },
  {
    "role": "assistant",
    "content": "Hi there! How can I help you?"
  }
]

Turn 3: user says: I want to learn about AI.
Current Conversation:
[
  {
    "role": "system",
    "content": "Summary of conversation:\nThe user greeted the assistant, the assistant responded with a friendly greeting and offered help, and the user expressed interest in learning about AI."
  }
]

Turn 4: assistant says: AI is the simulation of human intelligence in machines.
Current Conversation:
[
  {
    "role": "system",
    "content": "Summary of conversation:\nThe user greeted the assistant, the assistant responded with a friendly greeting and offered help, and the user expressed interest in learning about AI."
  },
  {
    "role": "assistant",
    "content": "AI is 

##Step 8: Truncation in Action

Finally, I show how we can cut the conversation history:  
- Only keep the last 3 turns  
- Or keep only up to 20 words  

This is proof that both truncation methods work.  


In [56]:
print("Truncate by Last 3 Turns")
truncated_turns = truncate_by_turns(conversation_history, 3)
print(json.dumps(truncated_turns, indent=2))
print("\nTruncate by Word Count (max 20 words)")
truncated_words = truncate_by_word_count(conversation_history, 20)
print(json.dumps(truncated_words, indent=2))


Truncate by Last 3 Turns
[
  {
    "role": "system",
    "content": "Summary of conversation:\nThe user greeted the assistant, who replied warmly and offered help. The user expressed interest in learning about AI. The assistant explained that AI is the simulation of human intelligence in machines. When the user asked about machine learning, the assistant clarified that machine learning enables systems to learn from data."
  }
]

Truncate by Word Count (max 20 words)
[]


# Conclusion

In this notebook, I successfully built a **conversation manager** using Groq API.  
Here’s what I achieved:

- Tracked the entire chat history  
- Summarized the chat periodically (every k turns)  
- Provided truncation options (by turns and by word count)  
- Kept the implementation clean and easy to follow  

This solution is production-friendly because:
- It prevents conversations from getting too long  
- It saves tokens and cost by keeping only summaries  
- It makes the system efficient and smart  




# Task 2 – JSON Schema Classification & Information Extraction


In this task, we are going to extract important details from user chats using structured data. The idea is to take real conversations and automatically pull out specific information like:

- Name  
- Email  
- Phone number  
- Location  
- Age

We will use a JSON schema to define how this information should be extracted and validated.

We’ll also use OpenAI’s function calling feature with Groq API, allowing us to process these chats in a structured way. Don't worry if this sounds a bit technical – we’ll go through everything step-by-step with lots of explanations!



##Step 2: Connect to Groq API

Here we connect to the Groq API using the OpenAI-compatible client.  
This way, we can use Groq just like we’d use OpenAI.


In [57]:
!pip install openai
from openai import OpenAI
import json


In [58]:
import openai
import os
# Use Colab's user_data to securely store the API key
from google.colab import userdata
# Securely input your Groq API key
api_key = userdata.get("GROQ_API_KEY")
# Configure OpenAI client for Groq API
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=api_key
)

###Defining the JSON Schema

Now we’ll create a JSON schema that tells the model exactly what information we want to extract from the chat. This schema acts like a template or a guide.

We’ll ask for the following details:
1. **Name** – The person's name.
2. **Email** – Their email address.
3. **Phone** – Their phone number.
4. **Location** – Where they are from or currently live.
5. **Age** – Their age or approximate age.

The schema helps the model return consistent and structured data every time!


In [59]:
# Define the JSON schema for extracting required information
schema = {
    "name": "extract_user_info",
    "description": "Extract user details from chat messages.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The full name of the user."
            },
            "email": {
                "type": "string",
                "description": "The email address of the user."
            },
            "phone": {
                "type": "string",
                "description": "The phone number of the user."
            },
            "location": {
                "type": "string",
                "description": "The location of the user."
            },
            "age": {
                "type": "string",
                "description": "The age or approximate age of the user."
            }
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}
print("JSON schema defined successfully")


JSON schema defined successfully


###Function to Call the API and Extract Information

Let's write a Python function that takes a chat message and uses the model to extract information based on the schema we just defined.

The function will:
1. Send the user’s message to the API.
2. Ask the model to extract details according to our schema.
3. Return the structured output.

This way, every time we pass a chat, the model will understand it and give us the details we need!


In [60]:
def extract_information(user_message):
    try:
        response = client.chat.completions.create(
            model="openai/gpt-oss-120b",
            messages=[
                {"role": "system", "content": "You are an information extractor."},
                {"role": "user", "content": user_message}
            ],
            functions=[schema],
            function_call={"name": "extract_user_info"}
        )
        # Parse function call arguments
        function_args = response.choices[0].message.function_call.arguments
        return json.loads(function_args)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


###Testing the Function with Example Chats

Now let's see this in action! We’ll pass a few sample chats where users introduce themselves, share their contact information, and talk about their location or age.

We expect the model to parse this information and return it neatly organized according to the schema.

Let’s test it with 3 examples.


In [61]:
examples = [
    "Hi, I’m Venkat, my email is venkat@example.com, phone 9876543210, I live in Hyderabad, age 21.",
    "Hello, I’m Aarti. You can reach me at aarti123@gmail.com, call me at 9123456789. I stay in Mumbai. I’m 25 years old.",
    "My name is Rohit Sharma. Email: rohit.s@gmail.com. Phone: 9000011111. Currently in Delhi. I’m 30."
]
for i, chat in enumerate(examples, 1):
    print(f"\n Example {i} ")
    result = extract_information(chat)
    print(json.dumps(result, indent=2))



 Example 1 
{
  "age": "21",
  "email": "venkat@example.com",
  "location": "Hyderabad",
  "name": "Venkat",
  "phone": "9876543210"
}

 Example 2 
{
  "age": "25",
  "email": "aarti123@gmail.com",
  "location": "Mumbai",
  "name": "Aarti",
  "phone": "9123456789"
}

 Example 3 
{
  "age": "30",
  "email": "rohit.s@gmail.com",
  "location": "Delhi",
  "name": "Rohit Sharma",
  "phone": "9000011111"
}


###Validating the Extracted Data

Let's make sure the model’s output matches the schema. We expect all fields — name, email, phone, location, and age — to be filled.

If anything is missing or incorrect, it will be easier to debug and fix it right away.

Let's print out a confirmation message for each result.


In [62]:
# Check if all required fields are present
for i, chat in enumerate(examples, start=1):
    print(f"Validation for Example {i}:")
    extracted_info = extract_information(chat)
    if extracted_info:
        missing = [key for key in schema['parameters']['required'] if key not in extracted_info]
        if missing:
            print(f"Missing fields: {missing}")
        else:
            print("All required fields are extracted correctly!")
            print(json.dumps(extracted_info, indent=2))
    else:
        print("Extraction failed.")
    print("\n" + "="*50 + "\n")


Validation for Example 1:
All required fields are extracted correctly!
{
  "age": "21",
  "email": "venkat@example.com",
  "location": "Hyderabad",
  "name": "Venkat",
  "phone": "9876543210"
}


Validation for Example 2:
All required fields are extracted correctly!
{
  "age": "25",
  "email": "aarti123@gmail.com",
  "location": "Mumbai",
  "name": "Aarti",
  "phone": "9123456789"
}


Validation for Example 3:
All required fields are extracted correctly!
{
  "age": "30",
  "email": "rohit.s@gmail.com",
  "location": "Delhi",
  "name": "Rohit Sharma",
  "phone": "9000011111"
}




###Task 2 Complete


Defined a JSON schema for extracting user details.  
Used Groq's API with OpenAI function calling to parse real chat messages.  
Structured and validated the extracted data according to the schema.  

This structured approach ensures that even casual conversations can be processed into neat, usable data formats.

With this, Task 2 is complete!Next, we can push this notebook to GitHub and prepare the final submission.

Let me know when you're ready to proceed with uploading and submitting everything!
